# Search and Filter DataFrames in PySpark HW

Now it's time to put what you've learn into action with a homework assignment!

In case you need it again, here is the link to the documentation for the full list available function in pyspark.sql.functions library:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions


### First set up your Spark Session!
Alright so first things first, let's start up our pyspark instance.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# Install pyspark
!pip install pyspark
# import findspark
# findspark.init()
import os
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
You are working with 1 core(s)


In [ ]:
# To display ALL columns
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
path = '/content/drive/MyDrive/Data Science Intake43/5. Spark/spark-scripts/section2/Datasets/'
os.listdir(path)

['people.json',
 'zomato.csv',
 'youtubevideos.csv',
 'googleplaystore.csv',
 'fifa19.csv',
 'students.csv',
 'users1.parquet',
 'users3.parquet',
 'users2.parquet',
 'nyc_air_bnb.csv',
 'rec-crime-pfa.csv',
 'supermarket_sales.csv',
 'Rep_vs_Dem_tweets.csv',
 'pga_tour_historical.csv',
 'uw-madison-courses']

## Read in the DataFrame for this Notebook

We will be continuing to use the fifa19.csv file for this notebook. Make sure that you are writting the correct path to the file. 

In [ ]:
fifa = spark.read.csv(path+'fifa19.csv',inferSchema=True,header=True)

## About this dataframe

The **fifa19.csv** dataset includes a list of all the FIFA 2019 players and their attributes listed below: 

 - **General**: Age, Nationality, Overall, Potential, Club
 - **Metrics:** Value, Wage
 - **Player Descriptive:** Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight
 - **Possition:** LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, 
 - **Other:** Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

**Source:** https://www.kaggle.com/karangadiya/fifa19

Use the .toPandas() method to view the first few lines of the dataset so we know what we are working with. 

In [ ]:
# Take a look at the first few lines
fifa.limit(4).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5,4,4,Medium/ Medium,Messi,Yes,RF,10,"Jul 1, 2004",None,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5,4,5,High/ Low,C. Ronaldo,Yes,ST,7,"Jul 10, 2018",None,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5,5,5,High/ Medium,Neymar,Yes,LW,10,"Aug 3, 2017",None,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4,3,1,Medium/ Medium,Lean,Yes,GK,1,"Jul 1, 2011",None,2020,6'4,168lbs,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,€138.6M


Now print the schema of the dataset so we can see the data types of all the varaibles. 

In [ ]:
print(fifa.printSchema())

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

In [ ]:
# fifa df 
print((fifa.count(), len(fifa.columns)))

(18207, 89)


## Now let's get started!

### First things first..... import the pyspark sql functions library

Since we know we will be using it a lot.

In [ ]:
from pyspark.sql.functions import *

### 1. Select the Name and Position of each player in the dataframe

In [ ]:
fifa.select('Name', 'Position').show()

+-----------------+--------+
|             Name|Position|
+-----------------+--------+
|         L. Messi|      RF|
|Cristiano Ronaldo|      ST|
|        Neymar Jr|      LW|
|           De Gea|      GK|
|     K. De Bruyne|     RCM|
|        E. Hazard|      LF|
|        L. Modrić|     RCM|
|        L. Suárez|      RS|
|     Sergio Ramos|     RCB|
|         J. Oblak|      GK|
|   R. Lewandowski|      ST|
|         T. Kroos|     LCM|
|         D. Godín|      CB|
|      David Silva|     LCM|
|         N. Kanté|     LDM|
|        P. Dybala|      LF|
|          H. Kane|      ST|
|     A. Griezmann|     CAM|
|    M. ter Stegen|      GK|
|      T. Courtois|      GK|
+-----------------+--------+
only showing top 20 rows



### 1.1 Display the same results from above sorted by the players names

In [ ]:
fifa.select('Name', 'Position').orderBy('Name').show()

+--------------+--------+
|          Name|Position|
+--------------+--------+
|      A. Abang|      ST|
| A. Abdellaoui|      LB|
|  A. Abdennour|      CB|
|       A. Abdi|      CM|
| A. Abdu Jaber|      ST|
|A. Abdulhameed|      GK|
|  A. Abedzadeh|      GK|
|      A. Abeid|      LB|
|      A. Ablet|     LWB|
|    A. Abrashi|     CDM|
|   A. Abruscia|    null|
|    A. Absalem|      LB|
|    A. Accardi|      CB|
|    A. Acevedo|     RCB|
|     A. Acosta|      LB|
|     A. Acosta|      RM|
|     A. Acquah|     LCM|
|       A. Adam|      ST|
|      A. Addai|      ST|
|      A. Ademi|     RDM|
+--------------+--------+
only showing top 20 rows



### 2. Select only the players who belong to a club begining with FC

In [ ]:
fifa.select('Name', 'Club').where(fifa.Club.startswith('FC')).show(5, False)

+---------------+-----------------+
|Name           |Club             |
+---------------+-----------------+
|L. Messi       |FC Barcelona     |
|L. Suárez      |FC Barcelona     |
|R. Lewandowski |FC Bayern München|
|M. ter Stegen  |FC Barcelona     |
|Sergio Busquets|FC Barcelona     |
+---------------+-----------------+
only showing top 5 rows



### 3. Who is the oldest player in the dataset and how old are they?

Display only the name and age of the oldest player.

In [ ]:
fifa.select('Name', 'Age').orderBy('Age', ascending=False).show(1)

+--------+---+
|    Name|Age|
+--------+---+
|O. Pérez| 45|
+--------+---+
only showing top 1 row



### 4. Select only the following players from the dataframe:

 - L. Messi
 - Cristiano Ronaldo

In [ ]:
fifa.where(fifa.Name.isin('L. Messi', 'Cristiano Ronaldo')).show(5, False)

+---+------+-----------------+---+----------------------------------------------+-----------+-----------------------------------+-------+---------+------------+--------------------------------------------+-------+-----+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------------+
|_c0|ID    |Name             |Age|Photo                                         |Nation

### 5. Can you select the first character from the Release Clause variable which indicates the currency used?

In [ ]:
fifa.select('Name',fifa['Release Clause'].substr(0,1).alias('Release Currancy')).show(5)

+-----------------+----------------+
|             Name|Release Currancy|
+-----------------+----------------+
|         L. Messi|               €|
|Cristiano Ronaldo|               €|
|        Neymar Jr|               €|
|           De Gea|               €|
|     K. De Bruyne|               €|
+-----------------+----------------+
only showing top 5 rows



### 6. Can you select only the players who are over the age of 40?

In [ ]:
fifa.select('Name', 'Age').filter(fifa.Age > 40).show()

+-------------+---+
|         Name|Age|
+-------------+---+
|    J. Villar| 41|
|     B. Nivet| 41|
|     O. Pérez| 45|
|     C. Muñoz| 41|
|  S. Narazaki| 42|
| H. Sulaimani| 41|
|     M. Tyler| 41|
|    T. Warner| 44|
|K. Pilkington| 44|
+-------------+---+



In [ ]:
fifa.select('Name', 'Age').where(fifa.Age > 40).show()

+-------------+---+
|         Name|Age|
+-------------+---+
|    J. Villar| 41|
|     B. Nivet| 41|
|     O. Pérez| 45|
|     C. Muñoz| 41|
|  S. Narazaki| 42|
| H. Sulaimani| 41|
|     M. Tyler| 41|
|    T. Warner| 44|
|K. Pilkington| 44|
+-------------+---+



### That's is for now... Great Job!